In [11]:
import findspark
findspark.init()

In [12]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.getOrCreate()
import pandas as pd
import pyspark.sql.functions as F



In [14]:
#csv_file='parent_operator.csv'
#df_csv=spark.read.csv(csv_file)

In [15]:
parquet_file='agg_usage.parquet'
df_parquet=spark.read.parquet(parquet_file)

In [16]:
df_parquet.show()


+-------------------+------------------+-------------+--------------+-------------------+-----------------+------------+-----------+----------------------+----------------+--------------------+---------+---------+
|actual_call_dur_sec|call_direction_ind|call_type_key|charge_amt_rur|connection_type_key|location_type_key|network_type|num_of_call|parent_operator_code_b|roaming_type_key| rounded_data_volume| time_key|client_id|
+-------------------+------------------+-------------+--------------+-------------------+-----------------+------------+-----------+----------------------+----------------+--------------------+---------+---------+
|              115.0|                 1|            V|           0.0|                  3|                2|           G|        1.0|                   136|               H|   1.817427842689825|P20171025|  0728799|
|              122.0|                 2|            V|           0.0|                  3|                2|           G|        1.0|            

In [17]:
#df_parquet.groupBy('client_id').count().show()
#spark_df.filter("target LIKE 'good%'")
#col("target").like("good%"))
#df_parquet.groupBy('client_id').count().where(col("client_id").like("0737477%"))
#df_parquet.select('client_id',.sum('rounded_data_volume')).where(col("call_type_key").like ("G" or "Y" or "X")).show()
#SELECT `field` FROM `table` WHERE `field`='a' AND `field2`='b' AND `field3`='3'
#first= df_parquet.select('client_id','rounded_data_volume','call_type_key','roaming_type_key','network_type').filter(col("call_type_key")=="G" 

#first=df_parquet.select('client_id','rounded_data_volume','call_type_key','roaming_type_key','network_type').where(col("call_type_key") == 

In [18]:
df_parquet.createOrReplaceTempView("abonenti")


In [19]:
first_part = spark.sql("SELECT client_id,time_key, rounded_data_volume, call_type_key, roaming_type_key, network_type FROM abonenti WHERE call_type_key IN ('X','G','Y') AND roaming_type_key IN ('X','R','H') AND network_type =='L'")
first_part.show()

+---------+---------+--------------------+-------------+----------------+------------+
|client_id| time_key| rounded_data_volume|call_type_key|roaming_type_key|network_type|
+---------+---------+--------------------+-------------+----------------+------------+
|  0728799|P20171025|1.0806536452246535E8|            G|               H|           L|
|  0728799|P20171025|1.1561004545611225E8|            G|               H|           L|
|  0728799|P20171025|2.8808993796954613E7|            G|               H|           L|
|  0728799|P20171021|2.3781191623708323E8|            G|               H|           L|
|  0728799|P20171022|  1.97996695740181E7|            G|               H|           L|
|  0736727|P20171024| 2.278664858203568E8|            G|               H|           L|
|  0736727|P20171026| 1.074904873788039E9|            G|               H|           L|
|  0736727|P20171026|4.2750834632258606E8|            G|               H|           L|
|  0736727|P20171023| 4.085852294668086E8| 

In [20]:
first_part=first_part.withColumn('rounded_data_volume_MB', first_part['rounded_data_volume']/(1024*8*1024))
first_part.show()

+---------+---------+--------------------+-------------+----------------+------------+----------------------+
|client_id| time_key| rounded_data_volume|call_type_key|roaming_type_key|network_type|rounded_data_volume_MB|
+---------+---------+--------------------+-------------+----------------+------------+----------------------+
|  0728799|P20171025|1.0806536452246535E8|            G|               H|           L|    12.882395329769295|
|  0728799|P20171025|1.1561004545611225E8|            G|               H|           L|    13.781791383756667|
|  0728799|P20171025|2.8808993796954613E7|            G|               H|           L|    3.4342996832078234|
|  0728799|P20171021|2.3781191623708323E8|            G|               H|           L|     28.34938958133259|
|  0728799|P20171022|  1.97996695740181E7|            G|               H|           L|     2.360304543258917|
|  0736727|P20171024| 2.278664858203568E8|            G|               H|           L|    27.163801887077902|
|  0736727

In [21]:
first_part=first_part.groupBy("client_id","time_key").agg({"rounded_data_volume_MB":"sum"})


In [22]:
first_part.show()

+---------+---------+---------------------------+
|client_id| time_key|sum(rounded_data_volume_MB)|
+---------+---------+---------------------------+
|  0730208|P20171025|         13.517119580005573|
|  0725527|P20171021|         118.54035554473869|
|  0734188|P20171020|         294.24871142979015|
|  0733202|P20171021|          61.99598675542702|
|  0731839|P20171026|           812.061679931006|
|  0732443|P20171020|          33.47843185368929|
|  0719068|P20171025|         23.228999614379326|
|  0728027|P20171026|         198.96198129173283|
|  0725965|P20171025|          7.653962472265498|
|  0731845|P20171023|          9.143933833533183|
|  0734959|P20171021|         236.01714308680948|
|  0736365|P20171023|         295.41677265925324|
|  0737693|P20171021|          277.4954077749167|
|  0723598|P20171024|         445.05240455399786|
|  0723497|P20171022|       0.004215225709363607|
|  0729936|P20171025|          1.817427842689825|
|  0727882|P20171024|           4.60369203657917|


In [26]:
#ne_huinya = first_part.groupBy('client_id', 'time_key').agg({'rounded_data_volume' : 'sum'}).where((df_parquet.call_type_key == 'G' | df_parquet.call_type_key == 'Y' | df_parquet.call_type_key == 'X') & (df_parquet.roaming_type_key == 'X' | df_parquet.roaming_type_key == 'R' | df_parquet.roaming_type_key == 'H') & df_parquet.network_type == 'L')
#ne_huinya.show()

In [27]:
second_part = spark.sql("SELECT client_id,time_key, rounded_data_volume, call_type_key, roaming_type_key, network_type FROM abonenti WHERE call_type_key IN ('X','G','Y') AND roaming_type_key IN ('X','R','H')")
second_part=second_part.withColumn('rounded_data_volume_MB', second_part['rounded_data_volume']/(1024*8*1024))
second_part=second_part.groupBy("client_id","time_key").agg({"rounded_data_volume_MB":"sum"})


In [29]:
second_part.show()

+---------+---------+---------------------------+
|client_id| time_key|sum(rounded_data_volume_MB)|
+---------+---------+---------------------------+
|  0730208|P20171025|         13.517119580005573|
|  0725527|P20171021|         123.22502428573458|
|  0734188|P20171020|         358.49518500604785|
|  0733202|P20171021|          67.25503546281988|
|  0722239|P20171025|         120.07181360114589|
|  0731839|P20171026|           812.061679931006|
|  0732443|P20171020|          41.92885013103981|
|  0719068|P20171025|          23.34258885454744|
|  0728027|P20171026|         199.30896092380888|
|  0731590|P20171020|          1.337779527761187|
|  0725965|P20171025|         22.994721806532592|
|  0731845|P20171023|          9.143933833533183|
|  0734959|P20171021|         268.59773186440435|
|  0736365|P20171023|          304.1655845460298|
|  0737693|P20171021|          277.4954077749167|
|  0723598|P20171024|         445.05240455399786|
|  0723497|P20171022|        0.12734418721972163|


In [30]:
third_part = spark.sql("SELECT client_id,time_key, rounded_data_volume, call_type_key, roaming_type_key, network_type FROM abonenti WHERE call_type_key IN ('X','G','Y') AND roaming_type_key IN ('X','R','H') AND network_type =='G'")
third_part=third_part.withColumn('rounded_data_volume_MB', third_part['rounded_data_volume']/(1024*8*1024))
third_part=third_part.groupBy("client_id","time_key").agg({"rounded_data_volume_MB":"sum"})


In [31]:
third_part.show()

+---------+---------+---------------------------+
|client_id| time_key|sum(rounded_data_volume_MB)|
+---------+---------+---------------------------+
|  0725527|P20171021|          4.684668740995892|
|  0734188|P20171020|          64.24647357625778|
|  0733202|P20171021|          5.259048707392859|
|  0722239|P20171025|         120.07181360114589|
|  0732443|P20171020|          8.450418277350517|
|  0719068|P20171025|        0.11358924016811406|
|  0728027|P20171026|        0.34697963207603594|
|  0720438|P20171020|           9.93905851470998|
|  0731590|P20171020|          1.337779527761187|
|  0725965|P20171025|         15.340759334267092|
|  0734959|P20171021|          32.58058877759483|
|  0736365|P20171023|          8.748811886776519|
|  0723497|P20171022|        0.12312896151035801|
|  0723674|P20171022|          55.43154920203966|
|  0729936|P20171025|        0.11358924016811406|
|  0722165|P20171023|        0.09961243913180315|
|  0727882|P20171024|         0.0148642169751243|
